# Solana Calculations vs. Python

In [ ]:
#!pip install liualgotrader
#!pip install solana

import pandas as pd
import numpy as np
from scipy import stats
from typing import Tuple, List
from liualgotrader.common.data_loader import DataLoader
from liualgotrader.common.types import TimeScale
from datetime import datetime

from solana.keypair import Keypair
from solana.publickey import PublicKey
from solana.rpc.async_api import AsyncClient

from liualgotrader.common import solana as liusol

import matplotlib.pyplot as plt
%matplotlib inline

### Load Data for Analysis

In [ ]:
dl = DataLoader(scale=TimeScale.day)

In [ ]:
data = dl["TSLA"].close[-200:]

In [ ]:
data

### Calculate in Python

In [ ]:
def calc_trend(x: np.array, y: np.array) -> Tuple[float, float, float]:
    slope, intercept, r_value, _, _ = stats.linregress(x, y)
    
    return slope, intercept, r_value 

def f(x: int, slope: float, intercept: float) -> float:
    return slope * x + intercept

def py_trendline(data: pd.DataFrame) -> np.array:
    x = np.arange(len(data)) 
    y = data.to_numpy()
    
    slope, intercept, r_value = calc_trend(x, y)
    print(f"Python slope:{slope} intercept:{intercept}, r_value:{r_value} score:{slope * r_value * r_value}")
    return np.array([f(i, slope, intercept) for i in x])

In [ ]:
log_ts = np.log(data)
log2_ts = np.log2(data)

### Calculate in Solana

In [ ]:
def sol_trendline(slope:float, intercept:float, r_value:float, length:int) -> np.array:
    x = np.arange(length) 
    print(f"Solana slope:{slope} intercept:{intercept}, r_value:{r_value} score:{slope * r_value * r_value}")
    return np.array([f(i, slope, intercept) for i in x])

In [ ]:
solana_network: str = "https://api.devnet.solana.com"
programId: str = "64ZdvpvU73ig1NVd36xNGqpy5JyAN2kCnVoF7M4wJ53e"
wallet_filename = "/Users/amichayoren/.config/solana/id.json"

client = AsyncClient(solana_network)
print(f"Is connection established:{await client.is_connected()}")

In [ ]:
program_key = PublicKey(programId)
payer = liusol.load_wallet(wallet_filename)
response_key = await liusol.set_response_account(client, payer, program_key)

In [ ]:
data_sol = await liusol.get_program_result(
    client=client,
    payer=payer,
    program_key=program_key,
    response_key=response_key,
    data=data,
)
data_sol 

In [ ]:
data_sol_ln = await liusol.get_program_result(
    client=client,
    payer=payer,
    program_key=program_key,
    response_key=response_key,
    data=log_ts,
)
data_sol_ln

In [ ]:
data_sol_log2 = await liusol.get_program_result(
    client=client,
    payer=payer,
    program_key=program_key,
    response_key=response_key,
    data=log2_ts,
)
data_sol_log2

## Visualize Original data, Ln, Log2 w/ trend lines from both Python and Solana

In [ ]:
fig = plt.figure()
fig.set_figwidth(40)
fig.set_figheight(10)

lefty = plt.subplot(131)
                              
lefty.plot(
    data,
    label="200 Days close data",
)
lefty.plot(
    data.index,
    py_trendline(data),
    label="python trendline",
)
lefty.plot(
    data.index,
    sol_trendline(data_sol[0], data_sol[1], data_sol[2], len(data)),
    label="solana trendline",
)
lefty.legend()
plt.xticks(rotation=45)  

center = plt.subplot(132)

center.plot(
    log_ts,
    label="LOG (NL) 200 Days close data",
)
center.plot(
    data.index,
    py_trendline(log_ts),
    label="trendline",
)
center.plot(
    data.index,
    sol_trendline(data_sol_ln[0], data_sol_ln[1], data_sol_ln[2], len(log_ts)),
    label="solana trendline",
)
center.legend()
plt.xticks(rotation=45)  

righty = plt.subplot(133)
righty.plot(
    log2_ts,
    label="LOG2 200 Days close data",
)
righty.plot(
    data.index,
    py_trendline(log2_ts),
    label="trendline",
)
righty.plot(
    data.index,
    sol_trendline(data_sol_log2[0], data_sol_log2[1], data_sol_log2[2], len(log2_ts)),
    label="solana trendline",
)
righty.legend()
plt.xticks(rotation=45)  
plt.show()